<a href="https://colab.research.google.com/github/hayannn/AIFFEL_STUDY/blob/main/4_3_%E1%84%8B%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A1%E1%86%BC%E1%84%87%E1%85%B3%E1%86%AF%E1%84%92%E1%85%A1%E1%86%A8%E1%84%89%E1%85%B3%E1%86%B8%264_4_%E1%84%85%E1%85%A2%E1%86%AB%E1%84%83%E1%85%A5%E1%86%B7%E1%84%91%E1%85%A9%E1%84%85%E1%85%A6%E1%84%89%E1%85%B3%E1%84%90%E1%85%B3%264_5_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 앙상블 학습
### 앙상블 학습 개요
- 앙상블 학습 : 여러 개의 분류기(Classifier)를 생성하고 그 예측을 결합해 보다 정확한 최종 예측을 도출하는 기법
- 딥러닝은 비정형 데이터(이미지, 영상, 음성 등)에서 강력, 앙상블 학습은 정형 데이터가 강세

<br>

### 앙상블 학습의 유형
- `보팅(Voting)` : **서로 다른 알고리즘**을 가진 분류기를 결합
- `배깅(Bagging)` : 각각의 분류기가 **모두 같은 유형**의 알고리즘(ex. 랜덤 포레스트)
  - 부트스트래핑(Bootstrappng) : 개별 Classifier에게 데이터를 샘플링해서 추출하는 방식
  - 배깅 앙상블 : 개별 분류기가 부트스트래핑으로 샘플링된 데이터 세트에 대해 학습을 통해 개별 예측을 수행한 결과를 ➡️ 보팅을 통해 최종 예측 결과를 선정하는 방식
  - 중첩 허용
- `부스팅(Boosting)` : 여러 개 분류기가 순차적으로 학습 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서 올바르게 예측이 되도록 ➡️ 다음 분류기에 가중치를 부여하며 학습 및 예측 진행
  - 부스팅 모듈 : 그래디언트 부스트, XGBoost, LightGBM

- `스태킹` : 여러 가지 다른 모델의 예측 결과값을 다시 학습 데이터로 만들어서 다른 메타 모델로 재학습시켜 결과를 예측

<br>

### 보팅 유형 - 하드 보팅(Hard Voting)과 소프트 보팅(Soft Voting)
- 하드 보팅 : 다수결 원칙, 예측 결과값 중 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- **소프트 보팅** : 분류기들의 레이블 값 결정 확률을 모두 더하고 이를 평균해 이들 중 확률이 가장 높은 레이블 값을 최종 보팅 결과값으로 선정

> 주로 소프트 보팅이 예측 성능이 좋아 자주 사용



### 보팅 분류기(Voting Classifier)
- 로지스틱 회귀와 KNN을 기반으로 보팅 분류기 생성

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


- VotingClassifier : 주요 생성 인자로 estimators, voting 값 입력
  - estimators : 리스트 값으로 보팅에 사용될 여러 개의 Classifier 객체들을 **튜플** 형식으로 입력받음
  - voting : hard와 soft로 적용(기본은 hard)

In [ ]:
# 개별 모델은 로지스틱 회귀와 KNN
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    test_size=0.2 , random_state= 156)

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))

- 보팅 분류기의 정확도가 조금 높게 나타나긴 했으나, 보팅으로 여러 개의 기반 분류기를 결합한다고 해서 무조건 기반 분류기보다 예측 성능이 향상되지는 않는다!
- 전반적으로 다른 단일 ML 알고리즘보다 앙상블 학습이 뛰어난 예측 성능을 가지는 경우가 많다.

> 앙상블 학습은 결정 트리 알고리즘의 장점을 취하고 단점을 보완해 **편향-분산 트레이드오프**의 효과를 극대화할 수 있다!

# 04. 랜덤 포레스트(Random Forest)
#### 랜덤 포레스트의 개요 및 실습
- 랜덤 포레스트
  - 배깅 대표 예시
  - 기반 알고리즘 : 결정 트리
  - 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 하게 됨
  - **데이터가 중첩된 개별 데이터 세트에 결정 트리 분류기를 각각 적용**

<br>

- 앞서 사용한 데이터셋 사용

In [2]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [5]:
import pandas as pd

def get_human_dataset( ):

    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1108분류/data/human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()

    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1108분류/data/human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1108분류/data/human_activity/test/X_test.txt',sep='\s+', names=feature_name)

    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1108분류/data/human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1108분류/data/human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])

    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

<ipython-input-2-9eb5e13abc8b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[1] >0 else x[0] ,  axis=1)
<ipython-input-2-9eb5e13abc8b>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])


- 랜덤 포레스트 적용

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

# 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))

랜덤 포레스트 정확도: 0.9253


### 랜덤 포레스트 하이퍼 파라미터 및 튜닝
- `n_estimators`
  - 랜덤 포레스트에서 결정 트리 개수 지정
  - 디폴트 10개
  - 많이 설정할수록 좋은 성능 기대 -> 계속 증가시킨다고 성능이 무조건 향상되는 것은 아님(늘릴수록 학습 수행 시간 증가)

- `max_features`
  - 디폴트 auto(즉, sqrt와 같음)
  - 랜덤 포레스트 트리를 분할하는 피처 참조 시 전체 피처가 아닌 sqrt(전체 피처 개수)만큼 참조(전체 피처가 16개라면 분할을 위해 4개 참조)

- max_depth, min_samples_leaf, min_samples_split과 같은 파라미터도 적용 가능

In [7]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.9180


In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, \
                                 min_samples_split=8, random_state=0)
rf_clf1.fit(X_train , y_train)
pred = rf_clf1.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('rf_feature_importances_top20.tif', format='tif', dpi=300, bbox_inches='tight')

# 05. GBM(Gradient Boosting Machine)
- 여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가며 학습하는 방식
  - AdaBoost(Adaptive boosting)
    - 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적 알고리즘
  - 그래디언트 부스트


<br>

- GBM
  - 에이다부스트와 유사하지만, 가중치 업데이트를 **경사 하강법(Gradient Descent)**을 이용하는 것이 큰 차이
    - 오류값은 `실제값 - 예측값`
    - 오류식은 $h(x) = y - F(x)$
      - 분류 실제 결과값 : $y$, 피처를 $x_1, x_2,...,x_n$, 예측 함수 : $F(x)$
    - 이 오류식을 최소화하는 방향성을 가지고 반복적으로 가중치 값을 업데이트 하는 것이 바로 경사 하강법
    - 즉, **반복 수행을 통해 오류를 최소화할 수 있도록 가중치의 업데이트 값을 도출하는 방법**
  - 분류와 회귀 모두 가능
  - 사이킷런의 GradientBoostingClassifier 사용

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함(시작 시간 설정)
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

> 참고
- 아래는 책에 없지만, GridSearchCV로 GBM의 하이퍼 파라미터 튜닝을 수행하는 예제
- 사이킷런이 1.X로 업그레이드 되며서 GBM의 학습 속도가 현저하게 저하되는 문제가 오히려 발생
- **아래는 수행 시간이 오래 걸리므로 참고용으로만 사용(실행 X)**

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}
grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 predict 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

### GBM 하이퍼 파라미터 소개
- loss
  - 경사 하강법에서 사용할 비용 함수 지정
  - 대부분의 경우, 기본값인 'deviance' 사용

- learning_rate
  - GBM이 학습 진행을 할 때마다 적용하는 학습률
  - Weak learner가 순차적으로 오류값을 보정해 나가는데 적용하는 계수
  - 0~1 값, 기본값은 0.1

- n_estimators
  - weak learner의 개수
  - 기본값은 100
  - weak learner가 순차적으로 오류 보정 -> 개수가 많을수록 예측 성능이 일정 수준까지는 좋아짐
  - 개수가 많을수록 수행 시간이 오래 걸림

- subsample
  - weak learner가 학습에 사용하는 데이터 샘플링 비율
  - 기본값은 1
  - 전체 학습 데이터를 기반으로 학습한다는 의미(0.5이면 학습 데이터의 50%)
  - 과적합을 방지하기 위해서는 subsample을 1보다 작은 값으로 설정